In [1]:
%%HTML
<link rel="stylesheet" type="text/css" href="../custom.css">

# 応用計量分析２（第12回）
混合分布モデルとEMアルゴリズム






### 担当教員: 梶野 洸（かじの ひろし）


# EMアルゴリズムの実装


# 演習
混合ガウスモデルの場合、

- 内部状態
- 欲しい命令

は何か？

# 一つの答え

- 内部状態
    - 正規分布の平均、分散共分散行列 $\times K$個
    - 重み $\pi=\begin{bmatrix}\pi_1 &\dots & \pi_K \end{bmatrix}$
- 命令
    - データを入力して、パラメタを最尤推定する
    - データを入力して、各データの潜在変数を推定する（現状の内部状態を使って）

# 他の答え
補助的なものを持っていてもよい

- 内部状態
    - 正規分布の平均、分散共分散行列 $\times K$個
    - 重み $\pi=\begin{bmatrix}\pi_1 &\dots & \pi_K \end{bmatrix}$
    - __混合数 $K$ を陽に持っておくと何かと便利__
- 命令
    - データを入力して、パラメタを最尤推定する
        - __E ステップを実行する__
        - __M ステップを実行する__
        - __尤度を計算する__
    - データを入力して、各データの潜在変数を推定する（現状の内部状態を使って）

# 他の答え2
内部状態として他のクラスのオブジェクトを持っていてもよい

- 正規分布クラス
    - 内部状態
        - 平均
        - 分散共分散行列
    - 命令
        - データを入力して、パラメタを最尤推定する
        - データを入力して、尤度を計算する

- GMM クラス
    - 内部状態
        - 正規分布オブジェクト $\times K$個
        - 重み $\pi=\begin{bmatrix}\pi_1 &\dots & \pi_K \end{bmatrix}$
        - __混合数 $K$ を陽に持っておくと何かと便利__
    - 命令
        - データを入力して、パラメタを最尤推定する
            - __E ステップを実行する__
            - __M ステップを実行する__
            - __尤度を計算する__
        - データを入力して、各データの潜在変数を推定する（現状の内部状態を使って）

# Python での実装

まずは正規分布クラスを実装してみよう

# 課題

正規分布クラスを実装せよ（以前の資料からコピペでOK）

In [2]:
import numpy as np

class Gaussian:
    def __init__(self, dim):
        '''コンストラクタ（みたいなもの）
        オブジェクトを作るときに初めに実行される。
        内部状態の初期化に使う
        '''
        self.dim = dim
        '''
        self = オブジェクトを指す。 self.dim は、オブジェクトの dim という変数を指す。
        上の命令は、 self.dim に dim の値を代入することを表す
        '''
        self.set_mean(np.random.randn(dim)) # オブジェクトの mean という変数をランダムに初期化
        self.set_cov(np.identity(dim))
        
    def log_pdf(self, X):
        ''' 確率密度関数の対数を返す
        
        Parameters
        ----------
        X : numpy.array, shape (sample_size, dim)
        
        Returns
        -------
        log_pdf : array, shape (sample_size,)
        '''
        if X.shape[1] != self.dim: # 入力の形をチェックしています
            raise ValueError('X.shape must be (sample_size, dim)')
        return -0.5 * np.sum((X - self.mean) * (np.linalg.solve(self.cov, (X - self.mean).T).T), axis=1) \
            -0.5 * self.dim * np.log(2.0 * np.pi) - 0.5 * np.linalg.slogdet(self.cov)[1]

    def fit(self, X):
        ''' X を使って最尤推定をする
                
        Parameters
        ----------
        X : numpy.array, shape (sample_size, dim)
        '''
        if X.shape[1] != self.dim: # 入力の形をチェックしています
            raise ValueError('X.shape must be (sample_size, dim)')
        self.set_mean(np.mean(X, axis=0))
        self.set_cov((X - self.mean).T @ (X - self.mean) / X.shape[0])

    def sample(self, sample_size):
        ''' 現状のパラメタを使って `sample_size` のサイズのサンプルを生成する
        
        Parameters
        ----------------
        sample_size : int
        
        Returns
        -----------
        X : numpy.array, shape (sample_size, dim)
            各行は平均 `self.mean`, 分散 `self.cov` の正規分布に従う
        '''
        return np.random.multivariate_normal(self.mean, self.cov, size=sample_size)
        
    def set_mean(self, mean):
        if mean.shape != (self.dim,):
            raise ValueError('input shape inconsistency')
        self.mean = mean
    
    def set_cov(self, cov):
        if cov.shape != (self.dim, self.dim):
            raise ValueError('input shape inconsistency')
        if np.linalg.eigvalsh(cov)[0] <= 0:
            raise ValueError('covariance matrix must be positive semidefinite.')
        self.cov = cov

# 課題
- GMM クラスの `__init__` を書いてみよう

In [3]:
class GMM:
    def __init__(self, dim, num_components):
        self.dim = dim
        self.num_components = num_components

        self.weight = np.ones(self.num_components) / self.num_components
        self.gaussian_list = []
        for _ in range(self.num_components):
            self.gaussian_list.append(Gaussian(dim))

In [4]:
# gmm のオブジェクトができた
gmm = GMM(2, 10)
for each_gaussian in gmm.gaussian_list:
    print(each_gaussian.mean)
print(gmm.weight)

[-0.6205795   0.90844947]
[0.5084933  0.69319952]
[0.03627212 0.40881998]
[ 0.46472002 -0.12202306]
[-0.04780564 -0.76016656]
[-1.25989674 -0.34276315]
[ 0.32399017 -0.27069776]
[-1.31465383  0.49880164]
[-1.34773983  0.57573302]
[-0.77237177 -0.53951956]
[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]


以降、EMアルゴリズムでパラメタ推定をする `fit` を書きたい

- `log_pdf`: 現在のパラメタを用いてサンプルの対数尤度を計算するメソッド
- `e_step`: Eステップを実行するメソッド
- `m_step`: Mステップを実行するメソッド

の3つを実装し、これらを組み合わせて `fit` を書く

# 課題

- サンプルの対数尤度を計算するメソッド `log_pdf` を `GMM` に実装せよ
    - 引数: `X`: サンプルサイズ$\times$次元の行列
    - 返り値: $\begin{bmatrix}\log p(x_1; \theta) & \dots & \log p(x_N; \theta)\end{bmatrix}$ ($\theta$は現在の内部状態を用いる)
    $$
    p(x_n; \theta) = \sum_{k=1}^K \pi_{k} \mathcal{N}(x_n; \mu_k, \Sigma_k)
    $$

In [5]:
class GMM:
    def __init__(self, dim, num_components):
        self. dim = dim
        self.num_components = num_components
        self.gaussian_list = []
        self.weight = np.ones(self.num_components) / self.num_components
        for _ in range(self.num_components):
            self.gaussian_list.append(Gaussian(dim))

    def log_pdf(self, X):
        sample_size = X.shape[0]
        likelihood = np.zeros(sample_size) # 各データの尤度を計算する
        '''
        ここで現在の内部状態での likelihood を計算する
        
        ** ヒント **
        self.gaussian_list の各要素は Gaussian クラスのオブジェクト
        各データの対数尤度を計算する log_pdf という命令が実装してあった
        '''
        return np.log(likelihood) # 対数尤度が欲しいので対数をとる

# ヒント

- `self.gaussian_list[0].log_pdf(X)` はサンプル `X` のデータそれぞれに対する対数尤度を計算する
    - `X` が `sample_size` x `dim` とすると、返り値は長さ `sample_size` の配列

- `self.weight[0]` は $p(z=0; \theta)$ に対応する
- `self.weight[0] * np.exp(self.gaussian_list[0].log_pdf(X))` は `np.exp(self.gaussian_list[0].log_pdf(X))` の各要素に `self.weight[0]` を掛けた値を返す
    - `スカラー * 配列` と書くと、配列の各要素にスカラーを掛けてくれる（`numpy` の記法）

In [6]:
class GMM:
    def __init__(self, dim, num_components):
        self. dim = dim
        self.num_components = num_components
        self.gaussian_list = []
        self.weight = np.ones(self.num_components) / self.num_components
        for _ in range(self.num_components):
            self.gaussian_list.append(Gaussian(dim))

    def log_pdf(self, X):
        sample_size = X.shape[0]
        likelihood = np.zeros(sample_size) # 各データの尤度を計算する
        for each_component in range(self.num_components):
            likelihood = likelihood + self.weight[each_component] * np.exp(self.gaussian_list[each_component].log_pdf(X))
        return np.log(likelihood)

In [7]:
gmm = GMM(2, 10)
gmm.log_pdf(np.random.randn(100, 2)) # とりあえず何か計算はできている

array([-2.69652435, -2.96796248, -4.89131967, -3.1702481 , -3.24433021,
       -3.08248415, -2.91850573, -3.52285946, -3.16371418, -3.23237037,
       -3.74293894, -3.5214885 , -3.1583319 , -2.84891282, -2.75931064,
       -2.99683467, -3.72387996, -3.0640002 , -3.34914635, -4.06090585,
       -3.16345834, -3.41665388, -2.76468276, -3.2807772 , -3.66944035,
       -3.03921052, -3.33118624, -3.22770282, -2.94774037, -3.39416585,
       -3.1466523 , -5.50732003, -3.11963635, -3.65342442, -3.59434391,
       -3.70828337, -3.74232986, -3.0086893 , -3.23703287, -3.67535367,
       -2.81592221, -2.89318612, -4.05476207, -3.55411438, -3.09078557,
       -2.91188748, -3.44218755, -3.60437493, -3.54660401, -3.22791858,
       -3.27805091, -2.68483982, -3.24961246, -2.92229927, -3.33159244,
       -3.2882023 , -3.35939904, -4.06176047, -3.29525287, -2.68343194,
       -4.78424231, -2.83002678, -2.8115925 , -4.42558718, -3.10642393,
       -2.85641264, -3.06363619, -2.99696914, -2.96048535, -4.43

# 課題
- Eステップを実行するメソッド `e_step` を実装せよ
    - 引数: データ `X`: `sample_size` $\times$ `dim` の配列
    - 出力: $q(z_n)$
        - どのような形式で出力すべきか？

- $q(z_n)$ は $K$ 面サイコロだったので、$K$面サイコロの確率分布をデータの数だけ出力すれば良さそう
    - $K=$ `n_components`
-  `posterior`: `sample_size` $\times$ `n_components` の配列として出力
-  `posterior[n, k]` = $q(z_n=k)$

E-step: for each $n=1,\dots,N, k=1,\dots,K$,
$$q(z_n=k)\leftarrow \dfrac{\pi_k \mathcal{N}(x_n; \mu_k, \Sigma_k)}{\sum_{k=1}^K \pi_k \mathcal{N}(x_n; \mu_k, \Sigma_k)}$$

In [ ]:
class GMM:
    def __init__(self, dim, num_components):
        self.dim = dim
        self.num_components = num_components
        self.gaussian_list = []
        self.weight = np.ones(self.num_components) / self.num_components
        for _ in range(self.num_components):
            self.gaussian_list.append(Gaussian(dim))

    def log_pdf(self, X):
        sample_size = X.shape[0]
        likelihood = np.zeros(sample_size) # 各データの尤度を計算する
        for each_component in range(self.num_components):
            likelihood = likelihood + self.weight[each_component] * np.exp(self.gaussian_list[each_component].log_pdf(X))
        return np.log(likelihood)

    def e_step(self, X):
        sample_size = X.shape[0]
        posterior = np.zeros((sample_size, self.num_components))
        '''
        ここを埋める
        
        ** ヒント **
        各 k について、分母は共通
        1. 分子を先に計算して posterior に入れてしまう
        2. posterior[n, :].sum() は分母と等しくなる
        3. posterior[n, :] = posterior[n, :] / posterior[n, :].sum() とすれば良さそう
        '''
        return posterior

In [8]:
class GMM:
    def __init__(self, dim, num_components):
        self.dim = dim
        self.num_components = num_components
        self.gaussian_list = []
        self.weight = np.ones(self.num_components) / self.num_components
        for _ in range(self.num_components):
            self.gaussian_list.append(Gaussian(dim))

    def log_pdf(self, X):
        sample_size = X.shape[0]
        likelihood = np.zeros(sample_size) # 各データの尤度を計算する
        for each_component in range(self.num_components):
            likelihood = likelihood + self.weight[each_component] * np.exp(self.gaussian_list[each_component].log_pdf(X))
        return np.log(likelihood)

    def e_step(self, X):
        sample_size = X.shape[0]
        posterior = np.zeros((sample_size, self.num_components))
        # 各コンポーネントで対数尤度が計算できるので、それを利用
        for each_component in range(self.num_components):
            posterior[:, each_component] \
               = self.weight[each_component] * np.exp(self.gaussian_list[each_component].log_pdf(X))
        
        # まとめて正規化しているが、 for 文でやってもOK（遅くなるけど）
        posterior = posterior / posterior.sum(axis=1).reshape(-1, 1)
        return posterior

In [9]:
# q はデータがどちらのコンポーネントに近いかを表しているので、
# そんな感じの結果になって欲しい
gmm = GMM(2, 2)
X = gmm.gaussian_list[0].mean.reshape(1, 2)
print(gmm.e_step(X))
X = gmm.gaussian_list[1].mean.reshape(1, 2)
print(gmm.e_step(X))

[[0.88221534 0.11778466]]
[[0.11778466 0.88221534]]


# 課題
- Mステップを実行するメソッド `m_step` を実装せよ
    - 引数: 
        - データ `X`: `sample_size` $\times$ `dim` の配列
        - $q(z_n = k)$ : `sample_size` $\times$ `n_components`
    - 出力: なし

In [10]:
class GMM:
    def __init__(self, dim, num_components):
        self.dim = dim
        self.num_components = num_components
        self.gaussian_list = []
        self.weight = np.ones(self.num_components) / self.num_components
        for _ in range(self.num_components):
            self.gaussian_list.append(Gaussian(dim))
    
    def fit(self, X, eps=1e-8):
        sample_size = X.shape[0]
        converge = False
        old_ll = -np.inf
        new_ll = -np.inf
        while not converge:
            posterior = self.e_step(X)
            self.m_step(X, posterior)
            new_ll = self.log_pdf(X).sum()
            if new_ll < old_ll:
                raise ValueError('likelihood decreases!')
            if np.abs(old_ll - new_ll) / np.abs(new_ll) < eps:
                converge = True
            old_ll = new_ll
        return posterior
    
    def e_step(self, X):
        posterior = np.zeros((X.shape[0], self.num_components))
        for each_component in range(self.num_components):
            posterior[:, each_component] \
               = self.weight[each_component] * np.exp(self.gaussian_list[each_component].log_pdf(X))
        posterior = posterior / posterior.sum(axis=1).reshape(-1, 1)
        return posterior
    
    def m_step(self, X, posterior):
        self.weight = posterior.sum(axis=0)
        for each_component in range(self.num_components):
            self.gaussian_list[each_component].set_mean(posterior[:, each_component] @ X / self.weight[each_component])
            self.gaussian_list[each_component].set_cov(
                (posterior[:, each_component] * (X - self.gaussian_list[each_component].mean).T) \
                @ (X - self.gaussian_list[each_component].mean) / self.weight[each_component])
        self.weight = self.weight / np.sum(self.weight)
    
    def log_pdf(self, X):
        sample_size = X.shape[0]
        likelihood = np.zeros(sample_size) # 各データの尤度を計算する
        for each_component in range(self.num_components):
            likelihood = likelihood + self.weight[each_component] * np.exp(self.gaussian_list[each_component].log_pdf(X))
        return np.log(likelihood)

In [11]:
X = np.vstack((np.random.randn(100, 3), np.random.randn(100, 3) + 3))
gmm = GMM(3, 2)
posterior = gmm.fit(X)

In [12]:
print(gmm.gaussian_list[0].mean)
print(gmm.gaussian_list[1].mean)

[0.13980818 0.20565738 0.06642373]
[2.99866878 3.109732   3.04706779]


In [13]:
posterior

array([[9.99990558e-01, 9.44244297e-06],
       [1.00000000e+00, 8.36488013e-14],
       [1.00000000e+00, 1.35241168e-11],
       [9.99999973e-01, 2.74890763e-08],
       [9.99999529e-01, 4.71319607e-07],
       [9.99999968e-01, 3.15163883e-08],
       [9.99999945e-01, 5.49695989e-08],
       [1.00000000e+00, 3.82693458e-10],
       [1.00000000e+00, 2.61726889e-12],
       [9.99999816e-01, 1.84044374e-07],
       [9.99993365e-01, 6.63463824e-06],
       [1.00000000e+00, 6.62717623e-14],
       [9.99802783e-01, 1.97217288e-04],
       [9.99999999e-01, 1.31993665e-09],
       [9.46845549e-01, 5.31544509e-02],
       [9.99999026e-01, 9.73648515e-07],
       [1.00000000e+00, 1.56758489e-11],
       [9.99999945e-01, 5.51277759e-08],
       [1.00000000e+00, 5.14591345e-11],
       [9.99650417e-01, 3.49583058e-04],
       [9.99244383e-01, 7.55617086e-04],
       [9.99998068e-01, 1.93164393e-06],
       [9.99999988e-01, 1.16642477e-08],
       [9.69406813e-01, 3.05931874e-02],
       [1.000000